<a href="https://colab.research.google.com/github/leorehem/espm/blob/main/Aula_04_FUNDAMENTOS_DE_DATA_SCIENCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS DE DATA SCIENCE: VISUALIZAÇÃO DE DADOS, STORYTELLING E ESTATÍSTICA**

**prof: Sérgio Assunção Monteiro, DSc**

*   linkedin: https://www.linkedin.com/in/sergio-assun%C3%A7%C3%A3o-monteiro-b781897b/

*   lattes: http://lattes.cnpq.br/9489191035734025

*   github: https://github.com/sergiomonteiro76

**Fonte de dados:** https://www.kaggle.com/datasets/naniruddhan/online-advertising-digital-marketing-data

Os campos (colunas) da planilha são:

1. **month** - Mês dos dados (ex: April)  
2. **day** - Dia do mês (ex: 1, 2, ..., 6)  
3. **campaign_number** - Identificador da campanha (ex: camp 1, camp 2, camp 3)  
4. **user_engagement** - Nível de engajamento do usuário (High, Medium, Low)  
5. **banner** - Tamanho do banner (ex: 160 x 600, 240 x 400)  
6. **placement** - Posicionamento do banner (ex: abc, def, ghi)  
7. **displays** - Número de impressões (exibições) do banner  
8. **cost** - Custo associado às exibições  
9. **clicks** - Número de cliques no banner  
10. **revenue** - Receita gerada  
11. **post_click_conversions** - Conversões pós-clique  
12. **post_click_sales_amount** - Valor das vendas pós-clique  

Observação: Há colunas vazias ou com valores nulos no final de algumas linhas (representadas por `,,`), que podem ser ignoradas ou tratadas como ausentes.

# **1: Instalar e Importar Bibliotecas**

In [ ]:
!pip install plotly scipy statsmodels seaborn

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

# **2: Carregar e Limpar Dados**

In [ ]:
## Objetivo: Carregar o conjunto de dados, diagnosticar valores ausentes e prepará-lo para análise
import pandas as pd
import os

# Verificar se o arquivo existe
url = "/content/online_advertising_performance_data.csv"
if not os.path.exists(url):
    print("Erro: Arquivo não encontrado. Verifique se o arquivo foi carregado no Colab.")
    print("Instruções: Vá para a aba 'Arquivos' no Colab, clique em 'Fazer upload' e selecione o arquivo.")
else:
    print("Arquivo encontrado. Iniciando leitura...")

    # Ler o arquivo
    try:
        df = pd.read_csv(url, encoding='latin1', sep=',')
        print("Dados brutos carregados. Tamanho inicial:", df.shape)
        print("Primeiras linhas do DataFrame:\n", df.head())
        print("Colunas disponíveis:", df.columns.tolist())
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        df = None

    if df is not None:
        # Verificar valores ausentes em todas as colunas
        print("Valores ausentes por coluna:\n", df.isna().sum())

        # Remover colunas irrelevantes (Unnamed)
        cols_to_drop = [col for col in df.columns if 'Unnamed' in col]
        if cols_to_drop:
            print(f"Removendo colunas irrelevantes: {cols_to_drop}")
            df = df.drop(columns=cols_to_drop)

        # Verificar valores únicos em 'placement'
        print("Valores únicos em 'placement':", df['placement'].unique())

        # Limpeza dos dados
        # Remover linhas com 'placement' inválido (incluindo NaN)
        initial_rows = df.shape[0]
        df = df[df['placement'].notna() & (df['placement'] != '#N/A')]
        print(f"Linhas após remover '#N/A' e NaN em 'placement': {df.shape[0]} (removidas {initial_rows - df.shape[0]} linhas)")

        # Remover espaços em branco das colunas categóricas
        categorical_cols = ['month', 'campaign_number', 'user_engagement', 'banner', 'placement']
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].str.strip()

        # Converter colunas numéricas
        numeric_cols = ['displays', 'cost', 'clicks', 'revenue', 'post_click_conversions', 'post_click_sales_amount']
        for col in numeric_cols:
            if col in df.columns:
                initial_rows = df.shape[0]
                df[col] = pd.to_numeric(df[col], errors='coerce')
                print(f"Linhas com valores não numéricos em '{col}' após conversão:", df[col].isna().sum())

        # Criar coluna de data
        try:
            df['date'] = pd.to_datetime(df['month'] + ' ' + df['day'].astype(str), format='%B %d', errors='coerce')
            df['date'] = df['date'].apply(lambda x: x.replace(year=2025) if pd.notnull(x) else x)
            print("Linhas com datas inválidas:", df['date'].isna().sum())
        except Exception as e:
            print(f"Erro ao criar coluna de data: {e}")
            df['date'] = pd.NaT

        # Remover linhas com valores ausentes apenas nas colunas principais
        main_cols = numeric_cols + ['date', 'campaign_number', 'user_engagement', 'banner', 'placement']
        initial_rows = df.shape[0]
        df = df.dropna(subset=main_cols)
        print(f"Linhas após remover valores ausentes nas colunas principais: {df.shape[0]} (removidas {initial_rows - df.shape[0]} linhas)")

        if df.empty:
            print("Erro: DataFrame está vazio após a limpeza. Possíveis causas:")
            print("- Valores ausentes em colunas críticas (verifique a saída acima).")
            print("- Erros no formato de 'month' ou 'day'.")
            print("Sugestão: Inspecione os dados brutos e ajuste os parâmetros de limpeza.")
        else:
            print("Dados carregados e limpos com sucesso. Tamanho final:", df.shape)
            print("Primeiras linhas do DataFrame limpo:\n", df.head())

Arquivo encontrado. Iniciando leitura...
Dados brutos carregados. Tamanho inicial: (15408, 14)
Primeiras linhas do DataFrame:
    month  day campaign_number user_engagement     banner placement  displays  \
0  April    1          camp 1            High  160 x 600       abc         4   
1  April    1          camp 1            High  160 x 600       def     20170   
2  April    1          camp 1            High  160 x 600       ghi     14701   
3  April    1          camp 1            High  160 x 600       mno    171259   
4  April    1          camp 1             Low  160 x 600       def       552   

       cost  clicks   revenue  post_click_conversions  \
0    0.0060       0    0.0000                       0   
1   26.7824     158   28.9717                      23   
2   27.6304     158   28.9771                      78   
3  216.8750    1796  329.4518                     617   
4    0.0670       1    0.1834                       0   

   post_click_sales_amount  Unnamed: 12  Unnamed:

In [ ]:
df.head()

,month,day,campaign_number,user_engagement,banner,placement,displays,cost,clicks,revenue,post_click_conversions,post_click_sales_amount,date
0,April,1,camp 1,High,160 x 600,abc,4,0.0060,0,0.0000,0,0.0000,2025-04-01
1,April,1,camp 1,High,160 x 600,def,20170,26.7824,158,28.9717,23,1972.4602,2025-04-01
2,April,1,camp 1,High,160 x 600,ghi,14701,27.6304,158,28.9771,78,2497.2636,2025-04-01
3,April,1,camp 1,High,160 x 600,mno,171259,216.8750,1796,329.4518,617,24625.3234,2025-04-01
4,April,1,camp 1,Low,160 x 600,def,552,0.0670,1,0.1834,0,0.0000,2025-04-01


In [ ]:
df.tail()

,month,day,campaign_number,user_engagement,banner,placement,displays,cost,clicks,revenue,post_click_conversions,post_click_sales_amount,date
15403,April,1,camp 1,Low,160 x 600,ghi,16,0.0249,0,0.0000,0,0.0000,2025-04-01
15404,April,1,camp 1,Low,160 x 600,mno,2234,0.4044,10,1.8347,3,101.7494,2025-04-01
15405,June,29,camp 1,High,800 x 250,ghi,1,0.0157,0,0.0000,0,0.0000,2025-06-29
15406,June,29,camp 1,High,800 x 250,mno,4,0.0123,0,0.0000,0,0.0000,2025-06-29
15407,June,29,camp 3,High,240 x 400,def,1209,0.3184,2,0.1115,3,110.4224,2025-06-29


In [ ]:
df.describe() #Análise Exploratória Inicial dos Dados

,day,displays,cost,clicks,revenue,post_click_conversions,post_click_sales_amount,date
count,14995.000000,14995.000000,14995.000000,14995.000000,14995.000000,14995.000000,14995.000000,14995
mean,15.471957,15939.616939,11.683242,166.244215,18.423759,43.465555,2181.756135,2025-05-12 05:19:47.195731712
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2025-04-01 00:00:00
25%,8.000000,102.000000,0.030200,0.000000,0.000000,0.000000,0.000000,2025-04-19 00:00:00
50%,15.000000,1336.000000,0.377500,7.000000,0.542300,0.000000,0.000000,2025-05-10 00:00:00
75%,23.000000,9619.000000,2.705250,57.000000,4.000000,3.000000,185.275250,2025-06-04 00:00:00
max,31.000000,455986.000000,556.704800,14566.000000,2096.211600,3369.000000,199930.318000,2025-06-30 00:00:00
std,8.741317,44923.954378,45.950340,737.736875,98.059296,216.491678,10660.989934,NaN


# **3: Análise de Histogramas**

In [ ]:
## Objetivo: Visualizar a distribuição das principais métricas
fig = make_subplots(rows=2, cols=3, subplot_titles=numeric_cols)
for i, col in enumerate(numeric_cols, 1):
    row = (i-1)//3 + 1
    col_idx = (i-1)%3 + 1
    fig.add_trace(go.Histogram(x=df[col], nbinsx=50, name=col), row=row, col=col_idx)
fig.update_layout(title_text="Histogramas das Principais Métricas", height=600)
fig.show()

### Insights:
# - **Assimetria**: A maioria das métricas (e.g., `displays`, `cost`, `revenue`) é altamente assimétrica à direita, indicando muitas observações de baixo valor e poucos outliers de alto valor.
# - **Decisão**: Focar em campanhas com alto `revenue` e `post_click_sales_amount` para maximizar o ROI, pois essas métricas mostram grande variância.


# **4: Análise de Dispersão**

In [ ]:
## Objetivo: Explorar relações entre cliques, receita e custo
fig = px.scatter(df, x='clicks', y='revenue', color='campaign_number', size='cost',
                 hover_data=['banner', 'placement', 'user_engagement'],
                 title="Cliques vs Receita por Campanha")
fig.update_layout(xaxis_title="Cliques", yaxis_title="Receita")
fig.show()

### Insights:
# - **Correlação Forte**: Maior número de `clicks` geralmente corresponde a maior `revenue`, especialmente para a campanha 1.
# - **Eficiência de Custo**: Bolhas maiores (maior `cost`) com baixa `revenue` indicam campanhas ineficientes. Tomadores de decisão devem investigar campanhas com alto custo e baixa receita (e.g., certos placements `mno`).

# **5: Ajuste de Distribuição de Probabilidade**

In [ ]:
## Objetivo: Ajustar uma distribuição log-normal à receita
# Filtrar receitas não-zero para evitar problemas com log
revenue_nonzero = df[df['revenue'] > 0]['revenue']
shape, loc, scale = stats.lognorm.fit(revenue_nonzero, floc=0)
x = np.linspace(revenue_nonzero.min(), revenue_nonzero.max(), 100)
pdf = stats.lognorm.pdf(x, shape, loc, scale)

fig = go.Figure()
fig.add_trace(go.Histogram(x=revenue_nonzero, histnorm='probability density', nbinsx=50, name='Receita'))
fig.add_trace(go.Scatter(x=x, y=pdf, mode='lines', name='Ajuste Log-Normal'))
fig.update_layout(title="Distribuição de Receita com Ajuste Log-Normal", xaxis_title="Receita", yaxis_title="Densidade")
fig.show()

### Insights:
# - **Ajuste Log-Normal**: A receita segue uma distribuição log-normal, indicando que eventos de alta receita são raros, mas impactantes.
# - **Decisão**: Direcionar campanhas para segmentos de alto engajamento para aumentar a probabilidade de resultados de alta receita, já que a cauda da distribuição representa ganhos significativos.

# **6: Regressão Linear**

In [ ]:
## Objetivo: Prever receita com base em características
# Codificar variáveis categóricas
le_campaign = LabelEncoder()
le_engagement = LabelEncoder()
le_banner = LabelEncoder()
le_placement = LabelEncoder()

df['campaign_number_enc'] = le_campaign.fit_transform(df['campaign_number'])
df['user_engagement_enc'] = le_engagement.fit_transform(df['user_engagement'])
df['banner_enc'] = le_banner.fit_transform(df['banner'])
df['placement_enc'] = le_placement.fit_transform(df['placement'])

# Características e alvo
X = df[['displays', 'cost', 'clicks', 'post_click_conversions', 'user_engagement_enc', 'banner_enc', 'placement_enc']]
y = df['revenue']

# Ajustar modelo
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# Coeficientes
coef_df = pd.DataFrame({'Característica': X.columns, 'Coeficiente': model.coef_})
print("Coeficientes da Regressão:\n", coef_df)
print("R² Score:", r2_score(y, y_pred))

fig = px.scatter(x=y, y=y_pred, labels={'x': 'Receita Real', 'y': 'Receita Prevista'},
                 title="Regressão Linear: Receita Real vs Prevista")
fig.add_trace(go.Scatter(x=[y.min(), y.max()], y=[y.min(), y.max()], mode='lines', name='Ajuste Ideal'))
fig.show()

### Insights:
# - **Fatores-Chave**: Características como `clicks` e `post_click_conversions` provavelmente têm coeficientes altos, indicando forte influência na `revenue`.
# - **Decisão**: Otimizar campanhas para aumentar cliques e conversões, focando em banners e placements de alto desempenho.


Coeficientes da Regressão:
            Característica  Coeficiente
0                displays    -0.000235
1                    cost     0.035985
2                  clicks     0.064506
3  post_click_conversions     0.277601
4     user_engagement_enc     1.594886
5              banner_enc    -0.025874
6           placement_enc    -0.366817
R² Score: 0.8783388111123245


# **7: Análise de Séries Temporais com ARIMA**

In [ ]:
## Objetivo: Analisar tendências de receita e prever com ARIMA
# Agregar receita por data e campanha
ts_data = df.groupby(['date', 'campaign_number'])['revenue'].sum().unstack().fillna(0)

# Plotar série temporal
fig = go.Figure()
for campaign in ts_data.columns:
    fig.add_trace(go.Scatter(x=ts_data.index, y=ts_data[campaign], mode='lines', name=campaign))
fig.update_layout(title="Séries Temporais de Receita por Campanha", xaxis_title="Data", yaxis_title="Receita")
fig.show()

# Previsão ARIMA para campanha 1
ts_camp1 = ts_data['camp 1'].resample('D').sum().fillna(0)
model_arima = ARIMA(ts_camp1, order=(5,1,0)).fit()
forecast = model_arima.forecast(steps=7)

# Plotar previsão
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts_camp1.index, y=ts_camp1, mode='lines', name='Real'))
fig.add_trace(go.Scatter(x=pd.date_range(ts_camp1.index[-1], periods=8, freq='D')[1:], y=forecast,
                         mode='lines', name='Previsão'))
fig.update_layout(title="Previsão ARIMA de Receita para Campanha 1", xaxis_title="Data", yaxis_title="Receita")
fig.show()

### Insights:
# - **Tendências**: A campanha 1 mostra maior volatilidade de receita em comparação com outras, com possíveis padrões sazonais.
# - **Decisão**: Usar previsões ARIMA para planejar alocação de orçamento na próxima semana, focando em campanhas com tendências estáveis ou crescentes.


# **8: Decomposição de Séries Temporais**

In [ ]:
## Objetivo: Decompor série temporal para entender componentes
decomposition = seasonal_decompose(ts_camp1, model='additive', period=7)
fig = make_subplots(rows=3, cols=1, subplot_titles=['Tendência', 'Sazonalidade', 'Resíduos'])
fig.add_trace(go.Scatter(x=ts_camp1.index, y=decomposition.trend, mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=ts_camp1.index, y=decomposition.seasonal, mode='lines'), row=2, col=1)
fig.add_trace(go.Scatter(x=ts_camp1.index, y=decomposition.resid, mode='lines'), row=3, col=1)
fig.update_layout(title="Decomposição de Séries Temporais para Receita da Campanha 1", height=800)
fig.show()

### Insights:
# - **Sazonalidade**: Sazonalidade semanal fraca sugere que campanhas podem ser alinhadas com dias de alta receita.
# - **Decisão**: Programar campanhas de alto orçamento durante períodos sazonais de pico para maximizar o retorno.

# **9: Análise de Segmentação**

In [ ]:
## Objetivo: Comparar desempenho entre segmentos
# Calcular métricas
df['CTR'] = df['clicks'] / df['displays'] * 100
df['CPC'] = df['cost'] / df['clicks'].replace(0, np.nan)
df['conversion_rate'] = df['post_click_conversions'] / df['clicks'].replace(0, np.nan) * 100

# Agregar por campanha, engajamento e banner
seg_data = df.groupby(['campaign_number', 'user_engagement', 'banner'])[['CTR', 'CPC', 'conversion_rate']].mean().reset_index()

fig = px.bar(seg_data, x='campaign_number', y='CTR', color='user_engagement', barmode='group',
             facet_col='banner', title="CTR por Campanha, Engajamento e Banner")
fig.show()

### Insights:
# - **Alto Engajamento**: Segmentos de alto `user_engagement` apresentam consistentemente maior CTR em todas as campanhas e banners.
# - **Decisão**: Alocar mais orçamento para segmentos de alto engajamento e banners maiores (e.g., 728x90) para melhores taxas de cliques.


# **10: Avaliação de Risco**

In [ ]:
## Objetivo: Avaliar risco das campanhas usando a razão custo/receita
df['cost_to_revenue'] = df['cost'] / df['revenue'].replace(0, np.nan)
risk_data = df.groupby(['campaign_number', 'banner'])['cost_to_revenue'].mean().unstack().fillna(0)

fig = px.imshow(risk_data, title="Mapa de Calor da Razão Custo/Receita", color_continuous_scale='RdYlGn')
fig.update_layout(xaxis_title="Banner", yaxis_title="Campanha")
fig.show()

### Insights:
# - **Alto Risco**: Campanhas com altas razões custo/receita (e.g., certos placements `468x60`) são menos eficientes.
# - **Decisão**: Reduzir investimento em banners e campanhas de alto risco ou otimizar seu direcionamento para reduzir custos.

# **11: Mineração de Dados e Análise de Correlação**

In [ ]:
## Objetivo: Identificar correlações entre características e eficácia
corr_matrix = df[numeric_cols].corr()
fig = px.imshow(corr_matrix, title="Mapa de Calor de Correlação das Características Numéricas", color_continuous_scale='RdBu')
fig.show()

# Gráfico de dispersão para interação entre características
fig = px.scatter(df, x='displays', y='clicks', color='user_engagement', size='revenue',
                 title="Impressões vs Cliques por Engajamento do Usuário")
fig.show()

### Insights:
# - **Correlações Fortes**: `clicks` e `post_click_conversions` são altamente correlacionados com `revenue` (correlação > 0.9).
# - **Decisão**: Focar em aumentar impressões (`displays`) para segmentos de alto engajamento para impulsionar cliques e conversões, pois impactam diretamente a receita.

# **12: Fato Interessante**

In [ ]:
## Objetivo: Destacar um insight inesperado
# Encontrar banner com altas conversões, mas poucas impressões
banner_perf = df.groupby('banner')[['displays', 'post_click_conversions']].sum()
banner_perf['conv_per_display'] = banner_perf['post_click_conversions'] / banner_perf['displays']
print("Desempenho do Banner (Conversões por Impressão):\n", banner_perf.sort_values('conv_per_display', ascending=False))

### Insight:
# - **Banner Subutilizado**: O banner `580x400` tem uma alta taxa de conversão por impressão, mas relativamente poucas impressões em comparação com `728x90`.
# - **Decisão**: Aumentar `displays` para `580x400` para capitalizar sua alta eficiência de conversão.

Desempenho do Banner (Conversões por Impressão):
            displays  post_click_conversions  conv_per_display
banner                                                       
580 x 400   7189697                   31759          0.004417
240 x 400  65783420                  269865          0.004102
728 x 90   76217085                  170395          0.002236
300 x 250  54838393                  119790          0.002184
670 x 90    5504972                   10619          0.001929
160 x 600  28783739                   48785          0.001695
800 x 250      2124                       3          0.001412
468 x 60     695126                     550          0.000791
